# NOAH HURDAT2 data 

https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2019-042820.txt

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup

# definition of hurdat2 data source 

https://www.nhc.noaa.gov/data/hurdat/hurdat2-format-atlantic.pdf reference

In [3]:
# There are two types of lines of data in the new format:  the header line and the data lines. 
#     The format is comma delimited to maximize its ease in use.  The header line has the following format: 
#         EP032012,           CARLOTTA,     18, 
#         1234567890123456789012345678901234567 
#     EP (Spaces 1 and 2) – Basin   EP – Northeast Pacific,  CP – North Central Pacific, AL Atlantic Basin, 
#     03 (Spaces 3 and 4)       – ATCF cyclone number for that year  
#     2012 (Spaces 5-8, before first comma)     – Year  
#     CARLOTTA (Spaces 19-28, before second comma)     – Name, if available, or else “UNNAMED”  
#     18 (Spaces 34-36)     – Number of best track entries – rows – to follow  
#  The remaining rows of data in the new format are the data lines. These have the following format:  
# 20120616, 0100, L, HU, 15.8N,  96.9W,  90,  976,   40,   80,   70,   80,   30,   50,   50,   60,   15,   35,   25,   25, 
# 123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890 

# 2012 (Spaces 1-4) – Year  06 (Spaces 5-6) – Month  16 (Spaces 7-8, before 1st comma) – Day  
# 01 (Spaces 11-12) – Hours in UTC (Universal Time Coordinate)  00 (Spaces 13-14, before 2nd comma) – Minutes 
# L (Space 17, before 3rd comma) – Record identifier (see notes below)  
#       L – Landfall (center of system crossing a coastline)  P – Minimum in central pressure  
#       I – An intensity peak in terms of both pressure and maximum wind  S – Change of status of the system  
#       T – Provides additional detail on the track (position) of the cyclone 
# HU (Spaces 20-21, before 4th comma) – Status of system. Options are:  
#         TD – Tropical cyclone of tropical depression intensity (< 34 knots)  
#         TS – Tropical cyclone of tropical storm intensity (34-63 knots)  
#         HU – Tropical cyclone of hurricane intensity (> 64 knots)  
#         EX – Extratropical cyclone (of any intensity)  
#         SD – Subtropical cyclone of subtropical depression intensity (< 34 knots) 
#         SS – Subtropical cyclone of subtropical storm intensity (> 34 knots) 
#         LO – A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)  
#         DB – Disturbance (of any intensity)  
# 15.8 (Spaces 24-27) – Latitude  N (Space 28, before 5th comma) – Hemisphere – North or South  
# 96.9 (Spaces 31-35) – Longitude  W (Space 36, before 6th comma) – Hemisphere – West or East  
# 90 (Spaces 39-41, before 7th comma) – Maximum sustained wind (in knots)  
# 976 (Spaces 44-47, before 8th comma) – Minimum Pressure (in millibars)  
# 40 (Spaces 50-53, before 9th comma) – 34 kt wind radii maximum extent in northeastern quadrant (in nautical miles)  
# 80 (Spaces 68-71, before 12th comma) – 34 kt wind radii maximum extent in northwestern quadrant (in nautical miles)  
# 30 (Spaces 74-77, before 13th comma) – 50 kt wind radii maximum extent in northeastern quadrant (in nautical miles)  
# 50 (Spaces 80-83, before 14th comma) – 50 kt wind radii maximum extent in southeastern quadrant (in nautical miles)  
# 50 (Spaces 86-89, before 15th comma) – 50 kt wind radii maximum extent in southwestern quadrant (in nautical miles)  
# 60 (Spaces 92-95, before 16th comma) – 50 kt wind radii maximum extent in northwestern quadrant (in nautical miles)  
# 15 (Spaces 98-101, before 17th comma) – 64 kt wind radii maximum extent in northeastern quadrant (in nautical miles)  
# 35 (Spaces 104-107, before 18th comma) – 64 kt wind radii maximum extent in southeastern quadrant (in nautical miles) 
# 25 (Spaces 110-113, before 19th comma) – 64 kt wind radii maximum extent in southwestern quadrant (in nautical miles)  
# 25 (Spaces 116-119, before 20th comma) – 64 kt wind radii maximum extent in northwestern quadrant (in nautical miles)  
 

# read in hurdat2 csv and name columns with a range of numbers 

- skip initial space to clean status of system

In [4]:
hurdat2 = pd.read_csv('..\data\hurdat2-1851-2019-042820.txt', delimiter=',', names = range(21), skipinitialspace=True)

In [5]:
hurdat2.head(20)         

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,AL011851,UNNAMED,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18510625,0000,NaN,HU,28.0N,94.8W,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
2,18510625,0600,NaN,HU,28.0N,95.4W,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
3,18510625,1200,NaN,HU,28.0N,96.0W,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
4,18510625,1800,NaN,HU,28.1N,96.5W,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
5,18510625,2100,L,HU,28.2N,96.8W,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
6,18510626,0000,NaN,HU,28.2N,97.0W,70.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
7,18510626,0600,NaN,TS,28.3N,97.6W,60.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
8,18510626,1200,NaN,TS,28.4N,98.3W,60.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
9,18510626,1800,NaN,TS,28.6N,98.9W,50.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN


# pull the storm identifier rows down into columns with iterrows

In [6]:
# iterrows to get new columns for Storm and Name
storm = []
storm_id = None
storm_name = None
for index, row in hurdat2.iterrows():
    if row[0].strip().startswith('AL'):
        storm_id = row[0]
        storm_name = row[1]
    else:
        storm_data = row.tolist()
        storm_data.insert(0,storm_id)
        storm_data.insert(1,storm_name)
        storm.append(storm_data)

In [7]:
hurdat2=pd.DataFrame(storm, columns = 
            ("id",
            "name",
            "date",
            "hours_minutes",
            "record_identifier",
            "status_of_system",
            "latitude",
            "longitude",
            "maximum_sustained_wind_knots",
            "maximum_pressure",
            "34_kt_ne",
            "34_kt_se",
            "34_kt_sw",
            "34_kt_nw",
            "50_kt_ne",
            "50_kt_se",
            "50_kt_sw",
            "50_kt_nw",
            "64_kt_ne",
            "64_kt_se",
            "64_kt_sw",
            "64_kt_nw",
             "na"
            ))

In [8]:
hurdat2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51853 entries, 0 to 51852
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            51853 non-null  object 
 1   name                          51853 non-null  object 
 2   date                          51853 non-null  object 
 3   hours_minutes                 51853 non-null  object 
 4   record_identifier             1086 non-null   object 
 5   status_of_system              51853 non-null  object 
 6   latitude                      51853 non-null  object 
 7   longitude                     51853 non-null  object 
 8   maximum_sustained_wind_knots  51853 non-null  float64
 9   maximum_pressure              51853 non-null  float64
 10  34_kt_ne                      51853 non-null  float64
 11  34_kt_se                      51853 non-null  float64
 12  34_kt_sw                      51853 non-null  float64
 13  3

In [9]:
hurdat2.head()

,id,name,date,hours_minutes,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,...,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw,na
0,AL011851,UNNAMED,18510625,0000,NaN,HU,28.0N,94.8W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
1,AL011851,UNNAMED,18510625,0600,NaN,HU,28.0N,95.4W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
2,AL011851,UNNAMED,18510625,1200,NaN,HU,28.0N,96.0W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
3,AL011851,UNNAMED,18510625,1800,NaN,HU,28.1N,96.5W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN


# create new columns 'year' and 'month_day' from the date column (retaining original column)

In [10]:
# create year column
hurdat2['year'] = hurdat2.date.str[:4]
hurdat2

,id,name,date,hours_minutes,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,...,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw,na,year
0,AL011851,UNNAMED,18510625,0000,NaN,HU,28.0N,94.8W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851
1,AL011851,UNNAMED,18510625,0600,NaN,HU,28.0N,95.4W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851
2,AL011851,UNNAMED,18510625,1200,NaN,HU,28.0N,96.0W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851
3,AL011851,UNNAMED,18510625,1800,NaN,HU,28.1N,96.5W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51848,AL202019,SEBASTIEN,20191126,1200,NaN,EX,52.2N,9.3W,45.0,970.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019
51849,AL202019,SEBASTIEN,20191126,1800,NaN,EX,52.2N,8.9W,40.0,972.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019
51850,AL202019,SEBASTIEN,20191127,0000,NaN,EX,51.8N,8.2W,40.0,974.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019
51851,AL202019,SEBASTIEN,20191127,0600,NaN,EX,51.4N,6.0W,40.0,976.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019


In [11]:
# create month_day column
hurdat2['month_day'] = hurdat2.date.str[4:]
hurdat2

,id,name,date,hours_minutes,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,...,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw,na,year,month_day
0,AL011851,UNNAMED,18510625,0000,NaN,HU,28.0N,94.8W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851,0625
1,AL011851,UNNAMED,18510625,0600,NaN,HU,28.0N,95.4W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851,0625
2,AL011851,UNNAMED,18510625,1200,NaN,HU,28.0N,96.0W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851,0625
3,AL011851,UNNAMED,18510625,1800,NaN,HU,28.1N,96.5W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851,0625
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1851,0625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51848,AL202019,SEBASTIEN,20191126,1200,NaN,EX,52.2N,9.3W,45.0,970.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019,1126
51849,AL202019,SEBASTIEN,20191126,1800,NaN,EX,52.2N,8.9W,40.0,972.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019,1126
51850,AL202019,SEBASTIEN,20191127,0000,NaN,EX,51.8N,8.2W,40.0,974.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019,1127
51851,AL202019,SEBASTIEN,20191127,0600,NaN,EX,51.4N,6.0W,40.0,976.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2019,1127


# filter to 20 years of data and validate 

In [12]:
# Filter the data sets to years 1999 to 2019
hurdat2_recent=hurdat2[hurdat2['year'].isin(['1999','2000','2001','2002','2003','2004','2005','2006',
                    '2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'])]
hurdat2_recent.head()

,id,name,date,hours_minutes,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,...,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw,na,year,month_day
41147,AL011999,ARLENE,19990611,1800,NaN,TD,27.1N,58.1W,30.0,1010.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1999,0611
41148,AL011999,ARLENE,19990612,0000,NaN,TD,27.7N,57.4W,30.0,1010.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1999,0612
41149,AL011999,ARLENE,19990612,0600,NaN,TD,28.1N,57.3W,30.0,1010.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1999,0612
41150,AL011999,ARLENE,19990612,1200,NaN,TS,28.3N,57.3W,35.0,1009.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1999,0612
41151,AL011999,ARLENE,19990612,1800,NaN,TS,28.5N,57.4W,45.0,1008.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,1999,0612


In [13]:
# validate that the frame is intact with filtered rows
hurdat2_recent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10706 entries, 41147 to 51852
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            10706 non-null  object 
 1   name                          10706 non-null  object 
 2   date                          10706 non-null  object 
 3   hours_minutes                 10706 non-null  object 
 4   record_identifier             360 non-null    object 
 5   status_of_system              10706 non-null  object 
 6   latitude                      10706 non-null  object 
 7   longitude                     10706 non-null  object 
 8   maximum_sustained_wind_knots  10706 non-null  float64
 9   maximum_pressure              10706 non-null  float64
 10  34_kt_ne                      10706 non-null  float64
 11  34_kt_se                      10706 non-null  float64
 12  34_kt_sw                      10706 non-null  float64
 1

# drop NA column

In [14]:
hurdat2_recent=hurdat2_recent.drop(['na'], axis=1)
hurdat2_recent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10706 entries, 41147 to 51852
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            10706 non-null  object 
 1   name                          10706 non-null  object 
 2   date                          10706 non-null  object 
 3   hours_minutes                 10706 non-null  object 
 4   record_identifier             360 non-null    object 
 5   status_of_system              10706 non-null  object 
 6   latitude                      10706 non-null  object 
 7   longitude                     10706 non-null  object 
 8   maximum_sustained_wind_knots  10706 non-null  float64
 9   maximum_pressure              10706 non-null  float64
 10  34_kt_ne                      10706 non-null  float64
 11  34_kt_se                      10706 non-null  float64
 12  34_kt_sw                      10706 non-null  float64
 1

# get column list and reorder with year and month_day at beginning

In [15]:
cols = list(hurdat2_recent.columns.values)
print(cols)

['id', 'name', 'date', 'hours_minutes', 'record_identifier', 'status_of_system', 'latitude', 'longitude', 'maximum_sustained_wind_knots', 'maximum_pressure', '34_kt_ne', '34_kt_se', '34_kt_sw', '34_kt_nw', '50_kt_ne', '50_kt_se', '50_kt_sw', '50_kt_nw', '64_kt_ne', '64_kt_se', '64_kt_sw', '64_kt_nw', 'year', 'month_day']


In [16]:
hurdat2_recent = hurdat2_recent[['id', 'name', 'year', 'month_day', 'date', 'hours_minutes','record_identifier',
                'status_of_system', 'latitude', 'longitude', 'maximum_sustained_wind_knots', 'maximum_pressure', 
                '34_kt_ne', '34_kt_se', '34_kt_sw', '34_kt_nw', '50_kt_ne', '50_kt_se', '50_kt_sw', '50_kt_nw', 
                                 '64_kt_ne', '64_kt_se', '64_kt_sw', '64_kt_nw']]
hurdat2_recent.head()

,id,name,year,month_day,date,hours_minutes,record_identifier,status_of_system,latitude,longitude,...,34_kt_sw,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw
41147,AL011999,ARLENE,1999,0611,19990611,1800,NaN,TD,27.1N,58.1W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41148,AL011999,ARLENE,1999,0612,19990612,0000,NaN,TD,27.7N,57.4W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41149,AL011999,ARLENE,1999,0612,19990612,0600,NaN,TD,28.1N,57.3W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41150,AL011999,ARLENE,1999,0612,19990612,1200,NaN,TS,28.3N,57.3W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41151,AL011999,ARLENE,1999,0612,19990612,1800,NaN,TS,28.5N,57.4W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [17]:
# determine unique values of status of system after white space is trimmed
hurdat2_recent.status_of_system.unique()

array(['TD', 'TS', 'HU', 'EX', 'LO', 'SD', 'WV', 'SS', 'DB'], dtype=object)

# Filter to Hurricane data (status_of_system == HU)

In [18]:
# Filter to HU data
hurdat2_recent=hurdat2_recent[hurdat2_recent['status_of_system']=='HU']
hurdat2_recent.head()
# note white space was cleaned from these values when read in from .csv

,id,name,year,month_day,date,hours_minutes,record_identifier,status_of_system,latitude,longitude,...,34_kt_sw,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw
41186,AL031999,BRET,1999,0821,19990821,0000,NaN,HU,21.9N,94.5W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41187,AL031999,BRET,1999,0821,19990821,0600,NaN,HU,22.5N,94.7W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41188,AL031999,BRET,1999,0821,19990821,1200,NaN,HU,23.1N,94.9W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41189,AL031999,BRET,1999,0821,19990821,1800,NaN,HU,23.8N,95.0W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41190,AL031999,BRET,1999,0822,19990822,0000,NaN,HU,24.7N,95.1W,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [19]:
hurdat2_recent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421 entries, 41186 to 51790
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            2421 non-null   object 
 1   name                          2421 non-null   object 
 2   year                          2421 non-null   object 
 3   month_day                     2421 non-null   object 
 4   date                          2421 non-null   object 
 5   hours_minutes                 2421 non-null   object 
 6   record_identifier             171 non-null    object 
 7   status_of_system              2421 non-null   object 
 8   latitude                      2421 non-null   object 
 9   longitude                     2421 non-null   object 
 10  maximum_sustained_wind_knots  2421 non-null   float64
 11  maximum_pressure              2421 non-null   float64
 12  34_kt_ne                      2421 non-null   float64
 13

# which storms are in the filtered HU data set

In [20]:
# which storms are in the filtered HU data set
hurdat2_recent.name.unique()

array(['BRET', 'CINDY', 'DENNIS', 'FLOYD', 'GERT', 'IRENE', 'JOSE',
       'LENNY', 'ALBERTO', 'DEBBY', 'FLORENCE', 'GORDON', 'ISAAC',
       'JOYCE', 'KEITH', 'MICHAEL', 'ERIN', 'FELIX', 'GABRIELLE',
       'HUMBERTO', 'IRIS', 'KAREN', 'MICHELLE', 'NOEL', 'OLGA', 'GUSTAV',
       'ISIDORE', 'KYLE', 'LILI', 'CLAUDETTE', 'DANNY', 'ERIKA', 'FABIAN',
       'ISABEL', 'JUAN', 'KATE', 'ALEX', 'CHARLEY', 'DANIELLE', 'FRANCES',
       'GASTON', 'IVAN', 'JEANNE', 'KARL', 'LISA', 'EMILY', 'KATRINA',
       'MARIA', 'NATE', 'OPHELIA', 'PHILIPPE', 'RITA', 'STAN', 'VINCE',
       'WILMA', 'BETA', 'EPSILON', 'ERNESTO', 'HELENE', 'DEAN', 'LORENZO',
       'BERTHA', 'DOLLY', 'HANNA', 'IKE', 'OMAR', 'PALOMA', 'BILL',
       'FRED', 'IDA', 'EARL', 'IGOR', 'JULIA', 'OTTO', 'PAULA', 'RICHARD',
       'SHARY', 'TOMAS', 'KATIA', 'RINA', 'CHRIS', 'KIRK', 'LESLIE',
       'NADINE', 'RAFAEL', 'SANDY', 'INGRID', 'ARTHUR', 'CRISTOBAL',
       'EDOUARD', 'FAY', 'GONZALO', 'JOAQUIN', 'HERMINE', 'MATTHEW',
       

# Add Saffir Simpson hurricane wind scale Column 'hurricane_category' from 'maximum_sustained_wind_knots' column 

wind speeds less than 64 = " ", cat_1 < 83, cat_2 < 96, cat_3 < 113, cat_4 < 137 else cat_5 which is >= 137

In [21]:
# def f(row):
#     if row['A'] == row['B']:
#         val = 0
#     elif row['A'] > row['B']:
#         val = 1
#     else:
#         val = -1
#     return val
## added from Steven's notebook "None" instead of blank
# if wind['maximum_sustained_wind_knots'] < 64:
#         val = 'None'

def f(row):
    if row ['maximum_sustained_wind_knots'] < 64:
        val =0
    elif row['maximum_sustained_wind_knots'] <=82:
        val =1
    elif row['maximum_sustained_wind_knots'] <=95:
        val =2
    elif row['maximum_sustained_wind_knots'] <=112:
        val =3
    elif row['maximum_sustained_wind_knots'] <=136:
        val =4
    else:
        val =5
    return val    

In [22]:
# apply the itterows function  # df['C'] = df.apply(f, axis=1)

hurdat2_recent['hurricane_category'] = hurdat2_recent.apply(f, axis=1)

# confirm that the values for Saffir Simpson category are added

In [23]:
# confirm that the values for Saffir Simpson category are added
hurdat2_recent.hurricane_category.unique()

array([1, 2, 4, 3, 5], dtype=int64)

## reorder the columns with hurricane_category after name

In [24]:
hurdat2_recent = hurdat2_recent[['id', 'name', 'hurricane_category', 'year', 'month_day', 'date', 'hours_minutes','record_identifier',
                'status_of_system', 'latitude', 'longitude', 'maximum_sustained_wind_knots', 'maximum_pressure', 
                '34_kt_ne', '34_kt_se', '34_kt_sw', '34_kt_nw', '50_kt_ne', '50_kt_se', '50_kt_sw', '50_kt_nw', 
                                 '64_kt_ne', '64_kt_se', '64_kt_sw', '64_kt_nw']]
hurdat2_recent.head()

,id,name,hurricane_category,year,month_day,date,hours_minutes,record_identifier,status_of_system,latitude,...,34_kt_sw,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw
41186,AL031999,BRET,1,1999,0821,19990821,0000,NaN,HU,21.9N,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41187,AL031999,BRET,1,1999,0821,19990821,0600,NaN,HU,22.5N,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41188,AL031999,BRET,1,1999,0821,19990821,1200,NaN,HU,23.1N,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41189,AL031999,BRET,2,1999,0821,19990821,1800,NaN,HU,23.8N,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41190,AL031999,BRET,4,1999,0822,19990822,0000,NaN,HU,24.7N,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


## Write results to .csv Commented out until need to use again

In [25]:
# comment out after use until need again
# hurdat2_recent.to_csv('../data/hurdat2_recent.csv', index = False)

 data below was original exploratory

In [26]:
# String to be searched for AL in start of string  
### hurdat2_storms  = hurdat2.loc[hurdat2[0].str.startswith('AL', na=False)]

# one df IGNORES records with AL in start of string  
### hurdat2_detail=hurdat2.loc[~hurdat2[0].str.startswith(('AL'))]

From Steven Decker
Converting lat long to proper formatting

In [27]:
## From Steven Decker
# apparently it is conventional to have 'S' and 'W' as negative values
# Not exactly sure why it has to be exactly this way, used Stack Overflow
hurdat2_recent['latitude'] = hurdat2_recent['latitude'].map(lambda lat: lat[:-1] if lat[-1] == "N" else -lat[:-1])
hurdat2_recent['longitude']= hurdat2_recent['longitude'].map(lambda long: long[:-1] if long[-1] == "E" else "-" + long[:-1])

In [28]:
## From Steven Decker
# Convert to float
hurdat2_recent['latitude'] = hurdat2_recent['latitude'].astype(float)
hurdat2_recent['longitude'] = hurdat2_recent['longitude'].astype(float)

In [29]:
#review formatted lat and long
pd.set_option('display.max_columns', None)
hurdat2_recent.head()

,id,name,hurricane_category,year,month_day,date,hours_minutes,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,34_kt_ne,34_kt_se,34_kt_sw,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw
41186,AL031999,BRET,1,1999,0821,19990821,0000,NaN,HU,21.9,-94.5,65.0,983.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41187,AL031999,BRET,1,1999,0821,19990821,0600,NaN,HU,22.5,-94.7,75.0,980.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41188,AL031999,BRET,1,1999,0821,19990821,1200,NaN,HU,23.1,-94.9,80.0,979.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41189,AL031999,BRET,2,1999,0821,19990821,1800,NaN,HU,23.8,-95.0,90.0,975.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
41190,AL031999,BRET,4,1999,0822,19990822,0000,NaN,HU,24.7,-95.1,120.0,954.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


# Write to csv

In [30]:
# comment out after use until need again
hurdat2_recent.to_csv('../data/hurdat2_recent.csv', index = False)

## From Steven Decker counting hurricanes by month and year with date time conversion 

- new csv hurricanes_group20

In [31]:
# Replace the -999 values with NaN

# hurdat_al = hurdat_al.replace(-999, np.NaN)
hurdat2_recent = hurdat2_recent.replace(to_replace = -999, value=np.nan)

# Replace empty values with NaN
hurdat2_recent = hurdat2_recent.replace(to_replace='', value=np.nan)

In [40]:
# strip every value of the white space in front of it
hurdat2_recent[hurdat2_recent.select_dtypes(['object']).columns] = \
    hurdat2_recent.select_dtypes(['object']) \
        .apply(lambda x: x.str.strip())

In [41]:
# Convert to datetime
hurdat2['date'] = pd.to_datetime(hurdat2['date'])

In [42]:
# The column hours_minutes has a SPACE in front of it
# hurdat_al['hours_minutes'] = hurdat_al['hours_minutes'].str.strip()

In [43]:
hurdat2['date'] = hurdat2 \
    .apply(
        lambda time: time['date'].replace(hour=int(time['hours_minutes'][:2]), minute=int(time['hours_minutes'][2:])), 
        axis='columns'
    )

In [44]:
# Crosstab to examine grouped years and grouped months
pd.crosstab(hurdat2['date'].dt.year, hurdat2['date'].dt.month, rownames=['year'], colnames=['month'], 
margins=True)

month,1,2,3,4,5,6,7,8,9,10,11,12,All
year,,,,,,,,,,,,,
1851,0,0,0,0,0,14,2,49,16,17,0,0,98
1852,0,0,0,0,0,0,0,45,64,25,0,0,134
1853,0,0,0,0,0,0,0,10,74,16,0,0,100
1854,0,0,0,0,0,11,0,1,36,12,0,0,60
1855,0,0,0,0,0,0,0,24,11,0,0,0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,42,0,0,0,20,63,0,126,194,103,38,0,586
2017,0,0,0,27,0,29,22,126,292,93,21,0,610
2018,0,0,0,0,26,0,102,57,278,130,15,0,608


In [46]:
hurdat2_group20 = hurdat2_recent.groupby([hurdat2_recent['date'].dt.year, hurdat2_recent['date'].dt.month,
                                        hurdat2_recent['name']]).size()

In [48]:
hurdat2_group20.index = hurdat2_group20.index.set_names(['year', 'month', 'name'])

In [50]:
hurricanes_group20 = hurdat2_group20.reset_index().drop(0 , axis= 1)

In [51]:
# Do the same as above but ONLY with hurricanes
# Crosstab to examine grouped years and grouped months
pd.crosstab(hurricanes_group20['year'], hurricanes_group20['month'], rownames=['year'], colnames=['month'],
            margins=True)

month,1,6,7,8,9,10,11,12,All
year,,,,,,,,,
1999,0,0,0,3,2,2,1,0,8
2000,0,0,0,2,5,3,0,0,10
2001,0,0,0,0,4,2,3,0,9
2002,0,0,0,0,4,1,0,0,5
2003,0,0,2,2,4,1,0,0,9
2004,0,0,0,5,4,1,0,0,10
2005,0,0,3,2,5,4,0,1,15
2006,0,0,0,1,4,1,0,0,6
2007,0,0,0,1,4,0,1,0,6


## Write to CSV cross tab 20 year count

In [52]:
# comment out after use until need again
hurricanes_group20.to_csv('../data/hurricanes_group20.csv', index = False)